In [1]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.50.0


In [2]:
!nvidia-smi

Tue Nov 25 17:41:26 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   58C    P0              72W / 300W |  30216MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach0


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Dataset Name

In [5]:
dataset_name = "thenlpresearcher/in22_gen_eng_mar_hin"

In [6]:
from datasets import load_dataset
raw_datasets = load_dataset(dataset_name)

In [7]:
# -------------------- LOAD DATA --------------------
src_sentences = raw_datasets['test']["src"]
gt_mar     = raw_datasets['test']["mar_Deva"]
gt_hin    = raw_datasets['test']["hin_Deva"]

## Model Names

In [8]:
original_model_name = "ai4bharat/indictrans2-en-indic-dist-200M"
without_punct_model_name = "thenlpresearcher/iitb-en-indic-without-punct"
with_punct_model_name = "thenlpresearcher/iitb-en-indic-only-punct"
combined_x_punct_model_name = "thenlpresearcher/shalaka_fd_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"
combined_2x_punct_model_name = "thenlpresearcher/shalaka_indictrans2-en-indic-dist-200M_finetuned_eng_Latn_to_mar_Deva"

### Original

In [9]:
mode = "original"

In [10]:
model_name = original_model_name

In [11]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "mar_Deva"
tokenizer_name =  "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = src_sentences

import torch

def batch_translate(
    input_sentences,
    src_lang,
    tgt_lang,
    model,
    tokenizer,
    ip,
    device="cuda",
    batch_size=16,
):
    all_translations = []

    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i : i + batch_size]

        # Preprocess (handles entity mapping etc.)
        batch = ip.preprocess_batch(
            batch,
            src_lang=src_lang,
            tgt_lang=tgt_lang
        )

        # Tokenize on device
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        # Generate translations
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode
        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess (restore entities)
        decoded = ip.postprocess_batch(decoded, lang=tgt_lang)

        all_translations.extend(decoded)

    return all_translations

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

2025-11-25 17:41:33.772856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 17:41:33.834814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 17:41:35.848916: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
100%|███████████████████████████████████████| 32/32 [04:12<00:00,  7.88s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ. गुणधर्मांचा एक समूह असतो......................................................................................................................................................................................................................
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: महाराष्ट्रातील औरंगाबाद जिल्ह्यात असलेल्या अजिंठामध्ये, इ. स. पू. पहिल्या शतकापासून ते इ. स. पू. पाचव्या शतकापर्यंतच्या शिल्पांनी आणि चित्रांनी सुशोभित केलेल्या एकोणवीस चैत्य आणि विहार लेण्या आहेत..........................................

In [12]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ. गुणधर्मांचा एक समूह असतो.


In [13]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.processor import IndicProcessor
from tqdm import tqdm
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [02:46<00:00,  5.20s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: उपस्थिति सेवा करने वाले व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे उनके जूते, कपड़े, टाई, आभूषण, केशविन्यास, मेकअप, घड़ी, सौंदर्य प्रसाधन, इत्र आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: महाराष्ट्र के औरंगाबाद जिले में स्थित अजंता में उनतीस चैत्य और विहार गुफाएँ हैं जिन्हें पहली शताब्दी ईसा पूर्व से लेकर पाँचवीं शताब्दी ईस्वी तक की मूर्तियों और चित्रों से सजाया गया है।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: शरीर का रंग बाहरी रेखा के साथ मिल जाता है, जिससे आयतन का प्रभाव पैदा होता है।
eng_Latn: Ashoka started making extensive use of stone for sculptures and great monumen

In [14]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_hin = remove_prefix(translations, "hin_Deva eng_Latn ")
print(translations_hin[0])

उपस्थिति सेवा करने वाले व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे उनके जूते, कपड़े, टाई, आभूषण, केशविन्यास, मेकअप, घड़ी, सौंदर्य प्रसाधन, इत्र आदि।


In [15]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/original_outputs.csv


In [16]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [17]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 21.006437035029688, CHRF++: 54.22227351259379
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304


### Without Punctuation Model

In [18]:
mode = "without"

In [19]:
model_name = without_punct_model_name

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [21]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [02:58<00:00,  5.57s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn देखावा हा सेवा करणाऱ्या व्यक्तीशी संबंधित अनेक गुणधर्मांचा समूह आहे, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, रंगभूषा, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्रातील औरंगाबाद जिल्ह्यात वसलेल्या अजिंठामध्ये सामान्य युगाच्या पहिल्या शतकापासून ते सामान्य युगाच्या पाचव्या शतकापर्यंतच्या शिल्पे आणि चित्रांनी सुशोभित एकोणवीस, चैत्या आणि विहार लेण्या आहेत.......................................................................................................................................................................................................
eng_La

In [22]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

देखावा हा सेवा करणाऱ्या व्यक्तीशी संबंधित अनेक गुणधर्मांचा समूह आहे, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, रंगभूषा, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.


In [23]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [03:34<00:00,  6.70s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn रूप सेवा व्यक्ति से संबंधित गुणों का एक समूह है, जैसे कि बूट, कपडे, टाई, गहने, केशविन्यास, मेकअप, घडी, सौंदर्य प्रसाधन, अत्तर, आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्र के औरंगाबाद जिले में स्थित अजंता में उनतीस, चैत्य और विहार गुहां हैं, जो सामान्य युग पूर्व प्रथम शताब्दी से सामान्य युग पूर्व पांचवीं शताब्दी तक की मूर्तियों और चित्रों से सजी हुई हैं।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीर का रंग बाह्य रेखा के साथ विलीन हो जाता है, आयतन का प्रभाव पैदा होता है।
eng_Latn: Ashoka started making extensive use 

In [24]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/without_outputs.csv


In [25]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [26]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.0738889683598, CHRF++: 53.09188998251425
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304


### With Punctuation Model

In [27]:
mode = "with"

In [28]:
model_name = with_punct_model_name

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [30]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [03:34<00:00,  6.71s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn देखावा म्हणजे सेवेतील व्यक्तीशी संबंधित अनेक वैशिष्ट्यांचा समूह, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, रंगभूषा, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्रातील औरंगाबाद जिल्ह्यात वसलेल्या अजिंठामध्ये <ID1>. पहिल्या शतकापासून ते <ID1>. पाचव्या शतकापर्यंतच्या शिल्पे आणि चित्रांनी सुशोभित एकोणवीस चैत्या आणि विहार गुहा आहेत.
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीराचा रंग बाहेरील रेषेत विलीन होतो, ज्यामुळे आकारमानाचा प्रभाव तयार होतो. <p
eng_Latn: Ashoka started making extens

In [31]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

देखावा म्हणजे सेवेतील व्यक्तीशी संबंधित अनेक वैशिष्ट्यांचा समूह, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, रंगभूषा, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.


In [32]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [03:25<00:00,  6.41s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn रूप सेवा व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे कि बूट, कपडे, टाई, गहने, केशविन्यास, मेकअप, घडी, सौंदर्य प्रसाधन, अत्तर आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्र के औरंगाबाद जिले में स्थित अजिंठा में सामान्य युग पूर्व प्रथम शताब्दी से सामान्य युग पूर्व पांचवीं शताब्दी तक की मूर्तियों और चित्रों से सजी उनतीस चैत्या और विहार गुहाएं हैं।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीर का रंग बाह्य रेखा के साथ विलीन हो जाता है, आयतन का प्रभाव पैदा करता है।
eng_Latn: Ashoka started making extensive use of ston

In [33]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})


import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/with_outputs.csv


In [34]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [35]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 16.982325915302376, CHRF++: 53.23187987987722
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304


### Combined Punctuation Model (2x)

In [36]:
mode = "combined_2x"

In [37]:
model_name = combined_2x_punct_model_name

In [38]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [39]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [04:44<00:00,  8.89s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ...........................................................................................................................................................................................................
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्रातील औरंगाबाद जिल्ह्यात असलेल्या अजिंठामध्ये सामान्य युगपूर्व पहिल्या शतकापासून ते सामान्य युगपूर्व पाचव्या शतकापर्यंतच्या शिल्पे आणि चित्रांनी सुशोभित केलेल्या एकोणवीस चैत्या आणि विहार गुहा आहेत.
eng

In [40]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, टाय, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.


In [41]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [04:41<00:00,  8.80s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn एक रूप सेवा व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे कि उनके जूते, कपड़े, टाई, गहने, केशविन्यास, मेकअप, घड़ी, सौंदर्य प्रसाधन, इत्र, आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्र के औरंगाबाद जिले में स्थित अजंता में उनतीस चैत्य और विहार गुफाएं हैं जो सामान्य युग पूर्व पहली शताब्दी से लेकर सामान्य युग पांचवीं शताब्दी तक की मूर्तियों और चित्रों से सजी हुई हैं।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीर का रंग बाहरी रेखा के साथ विलीन हो जाता है, आयतन का प्रभाव पैदा करता है।
eng_Latn: Ashoka started making ext

In [42]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/combined_2x_outputs.csv


In [43]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [44]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.107875535858394, CHRF++: 53.502210011528476
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304


### Combined Punctuation Model (x)

In [45]:
mode = "combined_x"

In [46]:
model_name = combined_x_punct_model_name

In [47]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.float16, # performance might slightly vary for bfloat16
    attn_implementation="flash_attention_2"
).to(DEVICE)

In [48]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [04:50<00:00,  9.06s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ..........................................................................................................................................................................................................
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्रातील औरंगाबाद जिल्ह्यात असलेल्या अजिंठामध्ये एकोणवीस चैत्य आणि विहार लेणी आहेत, ज्या सामान्य युगपूर्व पहिल्या शतकापासून ते सामान्य युग पाचव्या शतकापर्यंतच्या शिल्पांनी आणि चित्रांनी सजविलेल्या आहेत.

In [49]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.


In [50]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [04:35<00:00,  8.61s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn एक उपस्थिति सेवा व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे कि उनके जूते, कपड़े, टाई, गहने, केशविन्यास, मेकअप, घड़ी, सौंदर्य प्रसाधन, इत्र, आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्र के औरंगाबाद जिले में स्थित अजंता में उनतीस चैत्य और विहार गुफाएं हैं जो ईसा पूर्व पहली शताब्दी से लेकर ईसा पूर्व पांचवीं शताब्दी तक की मूर्तियों और चित्रों से सजी हुई हैं।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीर का रंग बाहरी रेखा के साथ विलय हो जाता है, आयतन का प्रभाव पैदा करता है।
eng_Latn: Ashoka started making extensive

In [51]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/combined_x_outputs.csv


In [52]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [53]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 17.061257294836043, CHRF++: 53.36730681250599
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304


## T5

In [54]:
model_name = original_model_name
mode = "t5"

In [55]:
from transformers import pipeline

# Use batching + GPU (if available)
punctuator_pipeline = pipeline(
    "text2text-generation",
    model="thenlpresearcher/iitb-t5-finetuned-punctuation",
    device=device,            # GPU; use device=-1 for CPU
    batch_size=64        # adjust based on GPU RAM
)

def restore_punctuation_t5_batch(text_list):
    # The pipeline automatically batches under the hood
    outputs = punctuator_pipeline(
        text_list,
        max_length=128
    )
    # Pipeline returns list of dicts
    return [o["generated_text"] for o in outputs]

Device set to use cuda


In [56]:
# Run the whole batch in parallel
predicted_sentences = restore_punctuation_t5_batch(src_sentences)
print(len(predicted_sentences))

1024


In [57]:
input_sentences = predicted_sentences

In [58]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="mar_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [04:48<00:00,  9.02s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ..........................................................................................................................................................................................................
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra, has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn महाराष्ट्रातील औरंगाबाद जिल्ह्यात असलेल्या अजिंठामध्ये, <ID1>. पहिल्या शतकापासून ते <ID1>. पाचव्या शतकापर्यंतच्या शिल्पे आणि चित्रांनी सुशोभित केलेल्या एकोणवीस चैत्या आणि विहार लेण्या आहेत.
eng_Latn: Body co

In [59]:
import re
def remove_prefix(translations, prefix):
    ans = []
    for t in translations:
        t = t.strip()
        if t.startswith(prefix):
            t = t[len(prefix):]
        t = re.sub(r'\.+', '.', t)
        ans.append(t)
    return ans

translations_mar = remove_prefix(translations, "mar_Deva eng_Latn ")
print(translations_mar[0])

दिसणे म्हणजे सेवेतील व्यक्तीशी संबंधित गुणधर्मांचा एक समूह आहे, जसे की त्यांचे बूट, कपडे, बांधणी, दागिने, केशरचना, मेकअप, घड्याळ, सौंदर्यप्रसाधने, अत्तर इ.


In [60]:
translations = batch_translate(
    input_sentences,
    src_lang="eng_Latn",
    tgt_lang="hin_Deva",
    model=model,
    tokenizer=tokenizer,
    ip=ip,
    device=DEVICE,
    batch_size=32
)

for input_sentence, translation in zip(input_sentences[:10], translations[:10]):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

100%|███████████████████████████████████████| 32/32 [03:30<00:00,  6.59s/it]

eng_Latn: An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
mar_Deva: mar_Deva eng_Latn एक उपस्थिति सेवा व्यक्ति से संबंधित विशेषताओं का एक समूह है, जैसे कि उनके जूते, कपड़े, टाई, गहने, केशविन्यास, मेकअप, घड़ी, सौंदर्य प्रसाधन, इत्र, आदि।
eng_Latn: Ajanta, located in the Aurangabad District of Maharashtra, has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
mar_Deva: mar_Deva eng_Latn अजंता, जो महाराष्ट्र के औरंगाबाद जिले में स्थित है, में उनतीस चैत्य और विहार गुफाएँ हैं जो ईसा पूर्व पहली शताब्दी से लेकर ईसा पूर्व पांचवीं शताब्दी तक की मूर्तियों और चित्रों से सजाई गई हैं।
eng_Latn: Body colour gets merged with the outer line, creating the effect of volume.
mar_Deva: mar_Deva eng_Latn शरीर का रंग बाहरी रेखा के साथ विलय हो जाता है, आयतन का प्रभाव पैदा करता है।
eng_Latn: Ashoka started making 

In [61]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction_mar": translations_mar,
    "prediction_hin": translations_hin,
    "gt_mar": gt_mar,
    "gt_hin": gt_hin
})

import os
os.makedirs(dataset_name, exist_ok=True)
results_df.to_csv(f"{dataset_name}/{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {dataset_name}/{mode}_outputs.csv")

✔ Saved predictions to thenlpresearcher/in22_gen_eng_mar_hin/t5_outputs.csv


In [62]:
# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1] for r1 in ref1]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score_mar, chrf_score_mar = compute_scores(translations_mar, gt_mar)
bleu_score_hin, chrf_score_hin = compute_scores(translations_hin, gt_hin)

In [63]:
print('Metrics -- Marathi')
print(f"BLEU: {bleu_score_mar}, CHRF++: {chrf_score_mar}")

print('Metrics -- Hindi')
print(f"BLEU: {bleu_score_hin} CHRF++: {chrf_score_hin}")

Metrics -- Marathi
BLEU: 16.859264353373863, CHRF++: 53.18591413851399
Metrics -- Hindi
BLEU: 32.468323411697796 CHRF++: 57.78894029358304
